In [ ]:
import os
from collections import defaultdict
import math
import networkx as nx
import random
from tqdm import tqdm
from zipfile import ZipFile
from urllib.request import urlretrieve
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pubchempy import *
import networkx as nx
from sklearn.cluster import KMeans
import tensorflow
import keras
from keras.models import Model
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, Softmax
from keras.layers import Conv2D, GRU
from keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed, Masking, RepeatVector, Flatten, \
    Concatenate, Lambda
from keras.models import Model
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers, layers
from tensorflow.keras import regularizers
import tensorflow as tf


def next_step(graph, previous, current, p, q):
    neighbors = list(graph.neighbors(current))

    weights = []
    # Adjust the weights of the edges to the neighbors with respect to p and q.
    for neighbor in neighbors:
        if neighbor == previous:
            # Control the probability to return to the previous node.
            weights.append(graph[current][neighbor]["weight"] / p)
        elif graph.has_edge(neighbor, previous):
            # The probability of visiting a local node.
            weights.append(graph[current][neighbor]["weight"])
        else:
            # Control the probability to move forward.
            weights.append(graph[current][neighbor]["weight"] / q)

    # Compute the probabilities of visiting each neighbor.
    weight_sum = sum(weights)
    probabilities = [weight / weight_sum for weight in weights]
    # Probabilistically select a neighbor to visit.
    next = np.random.choice(neighbors, size=1, p=probabilities)[0]
    return next


def random_walk(graph, num_walks,vocabulary_lookup, num_steps, p, q):
    walks = []
    nodes = list(graph.nodes())
    # Perform multiple iterations of the random walk.
    for walk_iteration in range(num_walks):
        random.shuffle(nodes)

        for node in tqdm(
            nodes,
            position=0,
            leave=True,
            desc=f"Random walks iteration {walk_iteration + 1} of {num_walks}",
        ):
            # Start the walk with a random node from the graph.
            walk = [node]
            # Randomly walk for num_steps.
            while len(walk) < num_steps:
                current = walk[-1]
                previous = walk[-2] if len(walk) > 1 else None
                # Compute the next node to visit.
                next = next_step(graph, previous, current, p, q)
                walk.append(next)
            # Replace node ids (movie ids) in the walk with token ids.
            walk = [vocabulary_lookup[token] for token in walk]
            # Add the walk to the generated sequence.
            walks.append(walk)

    return walks


def generate_examples(sequences, window_size, num_negative_samples, vocabulary_size):
    example_weights = defaultdict(int)
    # Iterate over all sequences (walks).
    for sequence in tqdm(
        sequences,
        position=0,
        leave=True,
        desc=f"Generating postive and negative examples",
    ):
        # Generate positive and negative skip-gram pairs for a sequence (walk).
        pairs, labels = keras.preprocessing.sequence.skipgrams(
            sequence,
            vocabulary_size=vocabulary_size,
            window_size=window_size,
            negative_samples=num_negative_samples,
        )
        for idx in range(len(pairs)):
            pair = pairs[idx]
            label = labels[idx]
            target, context = min(pair[0], pair[1]), max(pair[0], pair[1])
            if target == context:
                continue
            entry = (target, context, label)
            example_weights[entry] += 1

    targets, contexts, labels, weights = [], [], [], []
    for entry in example_weights:
        weight = example_weights[entry]
        target, context, label = entry
        targets.append(target)
        contexts.append(context)
        labels.append(label)
        weights.append(weight)

    return np.array(targets), np.array(contexts), np.array(labels), np.array(weights)


def create_dataset(targets, contexts, labels, weights, batch_size):
    inputs = {
        "target": targets,
        "context": contexts,
    }
    dataset = tf.data.Dataset.from_tensor_slices((inputs, labels, weights))
    dataset = dataset.shuffle(buffer_size=batch_size * 2)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset




def create_model(vocabulary_size, embedding_dim):

    inputs = {
        "target": layers.Input(name="target", shape=(), dtype="int32"),
        "context": layers.Input(name="context", shape=(), dtype="int32"),
    }
    # Initialize item embeddings.
    embed_item = layers.Embedding(
        input_dim=vocabulary_size,
        output_dim=embedding_dim,
        embeddings_initializer="he_normal",
        embeddings_regularizer=keras.regularizers.l2(1e-6),
        name="item_embeddings",
    )
    # Lookup embeddings for target.
    target_embeddings = embed_item(inputs["target"])
    # Lookup embeddings for context.
    context_embeddings = embed_item(inputs["context"])
    # Compute dot similarity between target and context embeddings.
    logits = layers.Dot(axes=1, normalize=False, name="dot_similarity")(
        [target_embeddings, context_embeddings]
    )
    # Create the model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

def node2vec_on_PPI(protein_protein_graph):
    vocabulary_protein = ["NA"] + list(protein_protein_graph.nodes)
    vocabulary_lookup_protein = {token: idx for idx, token in enumerate(vocabulary_protein)}


    ##############################################

    # Random walk return parameter.
    p = 1
    # Random walk in-out parameter.
    q = 1
    # Number of iterations of random walks.
    num_walks = 3
    # Number of steps of each random walk.
    num_steps = 10

    walks_protein = random_walk(protein_protein_graph, num_walks,vocabulary_lookup_protein, num_steps, p, q)


    # print("Number of protein walks generated:", len(walks_protein))

    #################################################


    num_negative_samples = 10#4


    # generate examples for Protein-Protein Network

    targets_p, contexts_p, labels_p, weights_p = generate_examples(
      sequences=walks_protein,
      window_size=num_steps,
      num_negative_samples=num_negative_samples,
      vocabulary_size=len(vocabulary_protein),
    )
    batch_size = 1024

    dataset_protein = create_dataset(
      targets=targets_p,
      contexts=contexts_p,
      labels=labels_p,
      weights=weights_p,
      batch_size=batch_size,
    )


    learning_rate = 0.001
    protein_embeding_dim=200


    model_protein = create_model(len(vocabulary_protein), protein_embeding_dim)
    model_protein.compile(
      optimizer=keras.optimizers.Adam(learning_rate),
      loss=keras.losses.BinaryCrossentropy(from_logits=True),
    )

    #history_protein = model_protein.fit(dataset_protein, epochs=1)

    protein_embeddings = model_protein.get_layer("item_embeddings").get_weights()[0]
    # print("Protein Embeddings shape:", protein_embeddings.shape)
    return protein_embeddings



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 39.5 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for PubChemPy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13819 sha256=7951c12ed378cd1159087267ccc281151668e4b0dd19e172b8eb33b11a356cce
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built PubChemPy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: openpyxl
    Found existing installation: openpy

In [ ]:
import os
from collections import defaultdict
import math
import networkx as nx
import random
from tqdm import tqdm
from zipfile import ZipFile
from urllib.request import urlretrieve
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#import tensorflow_addons as tfa
from pubchempy import *
import networkx as nx
from sklearn.cluster import KMeans
import tensorflow
import keras
from keras.models import Model
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, Softmax
from keras.layers import Conv2D, GRU
from keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed, Masking, RepeatVector, Flatten, \
    Concatenate, Lambda
from keras.models import Model
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers, layers
from tensorflow.keras import regularizers
import tensorflow as tf




def get_list_of_proteins(path_dataset, dataset, idx = 1):
    
    data = pd.read_excel(path_dataset + dataset + '/protein-protein_network.xlsx')

    yy = data.iloc[:, 1]
    list_of_prots = yy.unique()

    yy = data.iloc[:, 0]
    tmp = yy.unique()
    list_of_prots = np.concatenate((list_of_prots, tmp))
    list_of_prots = np.unique(list_of_prots)

    data = pd.read_csv(path_dataset + dataset + '/drug_protein.csv')

    yy = data.iloc[:, 0]
    list_of_drugs = yy.unique()

    yy = data.iloc[:, 1]
    tmp = yy.unique()
    list_of_prots = np.concatenate((list_of_prots, tmp))
    list_of_prots = np.unique(list_of_prots)

    data = pd.read_csv(path_dataset + dataset + '/cell_protein.csv')

    yy = data.iloc[:, 2]
    tmp = yy.unique()
    list_of_prots = np.concatenate((list_of_prots, tmp))
    list_of_prots = np.unique(list_of_prots)
    return list_of_prots




def get_list_of_drugs(path_dataset, dataset):
    data = pd.read_csv(path_dataset + dataset + '/drug_protein.csv')

    yy = data.iloc[:, 0]
    list_of_drugs = yy.unique()

    data = pd.read_csv(path_dataset + dataset + '/drug_combinations.csv')
    if dataset == 'OncologyScreen':
      idx = 1
    else:
      idx = 3
    yy = data.iloc[:, idx]
    tmp = yy.unique()
    list_of_drugs = np.concatenate((list_of_drugs, tmp))
    list_of_drugs = np.unique(list_of_drugs)

    yy = data.iloc[:, idx+1]
    tmp = yy.unique()
    list_of_drugs = np.concatenate((list_of_drugs, tmp))

    # print(list_of_drugs)
    #list_of_drugs = np.unique(list_of_drugs)


    # print(list_of_drugs.shape)
    return list_of_drugs



def get_list_of_cells(path_dataset, dataset):

    data = pd.read_csv(path_dataset + dataset + '/cell_protein.csv')

    yy = data.iloc[:, 3]
    list_of_cells = yy.unique()

    data = pd.read_csv(path_dataset + dataset + '/drug_combinations.csv')
    if dataset == 'OncologyScreen':
      idx = 3
    else:
      idx = 2
    yy = data.iloc[:, idx]
    tmp = yy.unique()
    list_of_cells = np.concatenate((list_of_cells, tmp))
    list_of_cells = np.unique(list_of_cells)
    return list_of_cells



## Read drug smiles
CHARCANSMISET = {"#": 1, "%": 2, ")": 3, "(": 4, "+": 5, "-": 6,
                 ".": 7, "1": 8, "0": 9, "3": 10, "2": 11, "5": 12,
                 "4": 13, "7": 14, "6": 15, "9": 16, "8": 17, "=": 18,
                 "A": 19, "C": 20, "B": 21, "E": 22, "D": 23, "G": 24,
                 "F": 25, "I": 26, "H": 27, "K": 28, "M": 29, "L": 30,
                 "O": 31, "N": 32, "P": 33, "S": 34, "R": 35, "U": 36,
                 "T": 37, "W": 38, "V": 39, "Y": 40, "[": 41, "Z": 42,
                 "]": 43, "_": 44, "a": 45, "c": 46, "b": 47, "e": 48,
                 "d": 49, "g": 50, "f": 51, "i": 52, "h": 53, "m": 54,
                 "l": 55, "o": 56, "n": 57, "s": 58, "r": 59, "u": 60,
                 "t": 61, "y": 62, "@": 63, "/": 64, "\\": 0}


def label_smiles(line, MAX_SMI_LEN, smi_ch_ind):
    X = np.zeros(MAX_SMI_LEN)
    for i, ch in enumerate(line[:MAX_SMI_LEN]):  # x, smi_ch_ind, y
        X[i] = smi_ch_ind[ch]

    return X  # .tolist()


def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
    X = np.zeros(MAX_SEQ_LEN)

    for i, ch in enumerate(line[:MAX_SEQ_LEN]):
        X[i] = smi_ch_ind[ch]

    return X  # .tolist()


smiles_dict_len = 64
smiles_max_len = 100


def read_protein_protein_graph(path_dataset, dataset, list_of_prots):
    data = pd.read_excel(path_dataset + dataset + '/protein-protein_network.xlsx')

    protein_protein_graph = nx.Graph()

    for i in range(0, data.shape[0]):
        tmp = np.argwhere(list_of_prots == data.iloc[i, 0])[0]
        tmp1 = np.argwhere(list_of_prots == data.iloc[i, 1])[0]
        # data.iloc[i,0:2]
        # tmp=np.array(tmp)
        protein_protein_graph.add_edge(tmp[0], tmp1[0], weight=1)


    return protein_protein_graph


def read_protein_protein_matrix(path_dataset, dataset, list_of_prots, list_of_drugs):
    data = pd.read_csv(path_dataset + dataset + '/drug_protein.csv')
    indx_of_drugs_interaction = []
    indx_of_prots_interaction = []
    # print(list_of_prots.shape)
    protein_drug_matrix = np.zeros((list_of_prots.shape[0], list_of_drugs.shape[0]))
    for i in range(0, data.shape[0]):
        tmp = np.argwhere(list_of_drugs == data.iloc[i, 0])[0]
        # indx_of_drugs_interaction.append(tmp[0])

        tmp2 = np.argwhere(list_of_prots == data.iloc[i, 1])[0]
        # indx_of_prots_interaction.append(tmp[0])

        protein_drug_matrix[tmp2[0], tmp[0]] = 1
        # protein_interaction.append(data.iloc[i,1])


    return protein_drug_matrix



def read_protein_cell_matrix(path_dataset, dataset, list_of_prots, list_of_cells):
    data = pd.read_csv(path_dataset + dataset + '/cell_protein.csv')

    protein_cell_matrix = np.zeros((list_of_prots.shape[0], list_of_cells.shape[0]))
    for i in range(0, data.shape[0]):
        tmp = np.argwhere(list_of_prots == data.iloc[i, 2])[0]
        tmp2 = np.argwhere(list_of_cells == data.iloc[i, 3])[0]
        protein_cell_matrix[tmp[0], tmp2[0]] = 1

    return protein_cell_matrix


def read_drug_combinations_data(path_dataset, dataset, list_of_drugs, list_of_prots, list_of_cells):
    data = pd.read_csv(path_dataset + dataset + '/drug_combinations.csv')

    indx_of_drugs1_combinations = []
    indx_of_drugs2_combinations = []
    indx_of_cells_combinations = []
    drugs_combinations = []
    num_training_samples = data.shape[0]

    # print(sort_lbl.shape[0])
    #threshold1 = sort_lbl[17359]
    #threshold2 = sort_lbl[52077]
    if dataset == 'OncologyScreen':
      idx = 1
      idx_lbl = 4
      idx_cell = 3
    else:
      idx = 3
      idx_lbl = 5
      idx_cell = 2
    sort_lbl = np.sort(np.array(data.iloc[:, idx_lbl]))
    for i in range(0, data.shape[0]):
        # print(list_of_drugs)
        # print(data.iloc[i, idx])
        tmp = np.argwhere(list_of_drugs == data.iloc[i, idx])[0]
        indx_of_drugs1_combinations.append(tmp[0])
        tmp = np.argwhere(list_of_drugs == data.iloc[i, idx+1])[0]
        indx_of_drugs2_combinations.append(tmp[0])
        tmp = np.argwhere(list_of_cells == data.iloc[i, idx_cell])[0]
        indx_of_cells_combinations.append(tmp[0])
        """if data.iloc[i,5]<threshold1:
           drugs_combinations.append([1,0,0])
        if data.iloc[i,5]>threshold2:
          drugs_combinations.append([0,0,1])
        else:
          drugs_combinations.append([0,1,0])"""
        if data.iloc[i, idx_lbl] > np.float64(sort_lbl[int(len(sort_lbl)/2)]):  # (threshold1+threshold2)/2:
            drugs_combinations.append([1])
        else:
            drugs_combinations.append([0])


    return drugs_combinations, indx_of_drugs1_combinations, indx_of_drugs2_combinations, indx_of_cells_combinations


def apply_clustering_on_proteins(protein_embeddings):

    kmeans = KMeans(n_clusters=200, random_state=0).fit(protein_embeddings[0:15970, :])
    protein_embeddings = kmeans.cluster_centers_
    return protein_embeddings, kmeans


def update_protein_drug_matrix_by_clustering(protein_drug_matrix, kmeans):
    protein_drug_matrix_kmeans = np.zeros((200, protein_drug_matrix.shape[1]))
    for i in range(200):
        protein_drug_matrix_kmeans[i, :] = np.max(protein_drug_matrix[kmeans.labels_ == i, :], axis=0)
    protein_drug_matrix = protein_drug_matrix_kmeans
    return protein_drug_matrix



def update_protein_cell_matrix_by_clustering(protein_cell_matrix, kmeans):
    protein_cell_matrix_kmeans = np.zeros((200, protein_cell_matrix.shape[1]))
    for i in range(200):
        protein_cell_matrix_kmeans[i, :] = np.max(protein_cell_matrix[kmeans.labels_ == i, :], axis=0)
    protein_cell_matrix = protein_cell_matrix_kmeans
    return protein_cell_matrix



In [ ]:

import pandas
import os
from collections import defaultdict
import math
import networkx as nx
import random
from tqdm import tqdm
from zipfile import ZipFile
from urllib.request import urlretrieve
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#import tensorflow_addons as tfa
from pubchempy import *
import networkx as nx
from sklearn.cluster import KMeans
import tensorflow
import keras
from keras.models import Model
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, Softmax
from keras.layers import Conv2D, GRU
from keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed, Masking, RepeatVector, Flatten, \
    Concatenate, Lambda
from keras.layers import Dot, GlobalMaxPooling2D, Conv2D, Reshape
from keras.models import Model
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers, layers
from tensorflow.keras import regularizers
import tensorflow as tf
from sklearn.model_selection import KFold



path_dataset = 'set the path to the folders contain datasets'
dataset = 'OncologyScreen'



################################################################################
#        This part extract the protein protein graph and protein drug graph    #
################################################################################

list_of_prots = get_list_of_proteins(path_dataset, dataset)

list_of_drugs = get_list_of_drugs(path_dataset, dataset)

list_of_cells = get_list_of_cells(path_dataset, dataset)

protein_protein_graph = read_protein_protein_graph(path_dataset, dataset, list_of_prots)

protein_drug_matrix = read_protein_protein_matrix(path_dataset, dataset, list_of_prots, list_of_drugs)

protein_cell_matrix = read_protein_cell_matrix(path_dataset, dataset, list_of_prots, list_of_cells)

###################################################################################################
#     This part of code reads compound ID and get its SMILES sequence and save it as numpy array  #
#     For the OncologyScreen dataset and DrugCombDB dataset, smiles sequences are computed and it #
#     is available. For your custom dataset, you should comment out these lines to compute the    #
#     Smiles based on the their compound Ids.                                                     #
###################################################################################################
"""smiles_max_len = 200
smiles_unique1 = []
for i in range(0, list_of_drugs.shape[0]):
    # print(i)
    cs = get_compounds(list_of_drugs[i], 'name')
    c = Compound.from_cid(cs[0].cid)
    #smiles_unique1.append(label_smiles(c.canonical_smiles, smiles_max_len, CHARCANSMISET))
    smiles_unique1.append(c.canonical_smiles)
# print(list_of_drugs.shape[0])
np.save(path_dataset + dataset + '/smiles_sequence.npy',
        np.array(smiles_unique1))"""

########################################################################################################
#   Read the stored SMILES sequencs
smiles_unique = np.load(path_dataset + dataset + '/smiles_200.npy')

##########################################################################
# Compute similarity between MACCS fingerprint and morgan fingerprints
##########################################################################
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MACCSkeys
import pandas as pd
from molvs import standardize_smiles
import numpy as np
from rdkit import DataStructs

class MACCS:
    def __init__(self, smiles):
        self.smiles = smiles
        self.mols = Chem.MolFromSmiles(self.smiles)
        print(self.mols)
    def compute_MACCS(self):
        MACCS_list = []
        #header = ['bit' + str(i) for i in range(167)]
        ds = list(MACCSkeys.GenMACCSKeys(self.mols).ToBitString())
        MACCS_list.append(ds)
        return MACCS_list
        #df = pd.DataFrame(MACCS_list,columns=header)
        #df.insert(loc=0, column='smiles', value=self.smiles)
        #df.to_csv(name[:-4]+'_MACCS.csv', index=False)


smiles_sequence = np.load(path_dataset + dataset + '/smiles_sequence.npy')
maccs_fp = []
morgan_fp = []
for i in range(len(smiles_sequence)):
    maccs_descriptor = MACCS(standardize_smiles(smiles_sequence[i]))
    maccs_fp.append(np.int32(maccs_descriptor.compute_MACCS())[0])
    # Load the molecule from the SMILES string
    mol = Chem.MolFromSmiles(smiles_sequence[i])

    fp1 = AllChem.GetMorganFingerprintAsBitVect(mol, useChirality=True, radius=2, nBits=124)
    
    vec1 = np.array(fp1)
    morgan_fp.append(vec1)

def tanimoto_similarity(arr1, arr2):
    fp1 = DataStructs.CreateFromBitString(''.join(map(str, arr1)))
    fp2 = DataStructs.CreateFromBitString(''.join(map(str, arr2)))
    return DataStructs.FingerprintSimilarity(fp1, fp2, metric=DataStructs.TanimotoSimilarity)

tanimoto_sim_maccs = np.zeros((len(smiles_sequence), len(smiles_sequence)))
tanimoto_sim_morgan = np.zeros((len(smiles_sequence), len(smiles_sequence)))
for i in range(0,len(smiles_sequence)):
  for j in range(i,len(smiles_sequence)):
      tanimoto_sim_maccs[i,j] = tanimoto_similarity(maccs_fp[i], maccs_fp[j])
      tanimoto_sim_maccs[j,i] = tanimoto_sim_maccs[i,j]
      tanimoto_sim_morgan[i,j] = tanimoto_similarity(morgan_fp[i], morgan_fp[j])
      tanimoto_sim_morgan[j,i] = tanimoto_sim_morgan[i,j]
# Read drug combinations data
drugs_combinations, indx_of_drugs1_combinations, indx_of_drugs2_combinations, indx_of_cells_combinations = read_drug_combinations_data(path_dataset, dataset, list_of_drugs, list_of_prots, list_of_cells)

from numpy.linalg import norm
cell_similarities_ = np.zeros((len(list_of_cells),len(list_of_cells)))
for i in range(0, len(list_of_cells)):
  for j in range(i, len(list_of_cells)):
    cell_similarities_[i,j] = np.dot(protein_cell_matrix[:,i],protein_cell_matrix[:,j])/(norm(protein_cell_matrix[:,i])*norm(protein_cell_matrix[:,j]))
    cell_similarities_[j, i] = cell_similarities_[i,j]






# New Secti

In [ ]:
####################################################################
#          Define Auxiliary functions which are used in model       #
####################################################################

def dot_batch(inp):

    return tf.keras.backend.batch_dot(inp[0], inp[1], axes=(1, 2))  # tf.keras.activations.sigmoid()


def dot_batch_axs_pro(inp):

    return tf.keras.backend.batch_dot(inp[0], inp[1], axes=(2, 2))


def dot_batch_axs_drug(inp):

    return tf.keras.activations.sigmoid(tf.keras.backend.batch_dot(inp[0], inp[1], axes=(1, 1)))


def dot_batch_axs_cell(inp):

    return tf.keras.activations.sigmoid(
        tf.keras.backend.batch_dot(inp[0], inp[1], axes=(1, 1)))  # tf.math.multiply(inp[0],inp[1]))


def dot_batch_axs_toxic(inp):

    return tf.keras.backend.batch_dot(inp[0], inp[1], axes=(1, 1))

def compactness_loss(actual, features):
    features = Flatten()(features)
    k = 2000
    batch_size = 10
    dim = (batch_size, k)

    def zero(i):
        z = tf.zeros((1, dim[1]), dtype=tf.dtypes.float32)
        o = tf.ones((1, dim[1]), dtype=tf.dtypes.float32)
        arr = []
        for k in range(dim[0]):
            arr.append(o if k != i else z)
        res = tf.concat(arr, axis=0)
        return res

    masks = [zero(i) for i in range(batch_size)]
    m = (1 / (batch_size - 1)) * tf.map_fn(
        # row-wise summation
        lambda mask: tf.math.reduce_sum(features * mask, axis=0),
        masks,
        dtype=tf.float32,
    )
    dists = features - m
    sqrd_dists = tf.pow(dists, 2)
    red_dists = tf.math.reduce_sum(sqrd_dists, axis=1)
    compact_loss = (1 / (batch_size * k)) * tf.math.reduce_sum(red_dists)
    return compact_loss



def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=tf.expand_dims(images, axis=-1),
            sizes=[1, self.patch_size / 4, 1, 1],
            strides=[1, self.patch_size / 4, 1, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        num_patches = patches.shape[1]
        patches1 = patches[:,0:num_patches:2,:,:]
        patches2 = patches[:,2:num_patches:2,:,:]
        patches3 = patches[:,1:num_patches:2,:,:]
        patches4 = patches[:,3:num_patches:2,:,:]

        out1 = tf.concat((patches1[:,0:-1,:,:],patches2), axis = -1)
        out2 = tf.concat((patches3[:,0:-1,:,:],patches4), axis = -1)
        patches_1 = tf.concat((patches[:,0:-2,:,:],patches[:,1:-1,:,:]), axis = -1)

        patches_ = tf.concat((out1, out2), axis = 1)
        patches = tf.concat((patches_1, patches_), axis = 1)

        patch_dims = patches.shape[-1]
        # print(patches.shape)

        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded


def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    # augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


##############################################################
#           Feature extractor for Drugs                      #
##############################################################
# Hyperparameters
embedding_size = 20
num_filters = 64
protein_filter_lengths = 8
smiles_filter_lengths = 4
smiles_max_len = 200
protein_embeding_dim = 200


num_classes = 2
input_shape = (200, 1)

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 100
drug_size = 200  # We'll resize input sequences to this size
patch_size = 20  # Size of the patches to be extract from the input images
num_patches = 38*2#(drug_size // patch_size) * 4
projection_dim = 50
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [1024, 1024, 512]  # Size of the dense layers of the final classifier


METRICS = [
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR')  # precision-recall curve
]


num_prot_cluster = 200

# Define Shared Layers
Drug1_input = Input(shape=(smiles_max_len, 1), name='drug1_input', dtype='int32')  # dtype='int32',
Drug2_input = Input(shape=(smiles_max_len, 1), name='drug2_input', dtype='int32')  # dtype='int32',

Drug1_similarities = Input(shape=(len(smiles_sequence),1 ), name='drug1_similarities')
Drug2_similarities = Input(shape=(len(smiles_sequence),1 ), name='drug2_similarities')
cell_similarities = Input(shape=(len(list_of_cells),), name='cell_similarities')

# network for Drugs
smiles_embedding = Embedding(input_dim=65, output_dim=128, input_length=200)
smiles_first_conv = Conv1D(filters=32, kernel_size=3,  activation='relu', padding='same',  strides=1)
smiles_second_conv = Conv1D(filters=32*2, kernel_size=3,  activation='relu', padding='same',  strides=1)
smiles_third_conv = Conv1D(filters=32*3, kernel_size=3,  activation='relu', padding='same',  strides=1)
smiles_global_pooling = GlobalMaxPooling1D()

# Drug1 encoder

# network for Drug 1
# out1 = Embedding(input_dim=smiles_dict_len+1, output_dim = embedding_size, input_length=smiles_max_len,name='smiles_embedding') (Drug1_input)
patches = Patches(patch_size)(Drug1_input)
# Encode patches.
encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

#Input_encoded_patches = Input(shape = (76,50))
# Create multiple layers of the Transformer block.
for _ in range(transformer_layers):

    num_heads = 1
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=projection_dim, dropout=0.1
    )(encoded_patches, encoded_patches)
    print('--------------------------')
    print(encoded_patches.shape)
    print(attention_output.shape)
    print('***************************')
    # Skip connection 1.
    x2 = layers.Add()([attention_output, encoded_patches])
    # Layer normalization 2.
    x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
    # MLP.
    x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
    # Skip connection 2.
    encoded_patches = layers.Add()([x3, x2])
    print('--------------------------')
    print(encoded_patches.shape)
    print(x3.shape)
    print(x2.shape)
    print('***************************')



transformer_model = Model(inputs = [Drug1_input], outputs = [encoded_patches])
encode_drug1_ = transformer_model(Drug1_input)
encode_drug1 = layers.Flatten()(encode_drug1_[0])

num_classes = 2
input_shape = (200, 1)

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 100
drug_size = 200  # We'll resize input sequences to this size
patch_size = 20  # Size of the patches to be extract from the input images
num_patches = 38*2#(drug_size // patch_size) * 4
projection_dim = 50
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [512, 256, 128]

patches_2 = Patches(patch_size)(Drug2_input)
encoded_patches_2 = PatchEncoder(num_patches, projection_dim)(patches_2)

encode_drug2_ = transformer_model([Drug2_input])
encode_drug2 = layers.Flatten()(encode_drug2_[0])

"""encode_drug1 = smiles_embedding(Drug1_input)
encode_drug1 = smiles_first_conv(encode_drug1)
encode_drug1 = smiles_second_conv(encode_drug1)
encode_drug1_ = smiles_third_conv(encode_drug1)
encode_drug1 = smiles_global_pooling(encode_drug1_)

# Drug2 encoder
encode_drug2 = smiles_embedding(Drug2_input)
encode_drug2 = smiles_first_conv(encode_drug2)
encode_drug2 = smiles_second_conv(encode_drug2)
encode_drug2_ = smiles_third_conv(encode_drug2)
encode_drug2 = smiles_global_pooling(encode_drug2_)"""

sim_mat = Dot(axes=(2,2))([encode_drug1_[0], encode_drug2_[0]])
sim_mat = Reshape((76, 76, 1))(sim_mat)
sim_mat_first_conv = Conv2D(filters=32, kernel_size=(3, 3),  activation='relu', padding='same',  strides=1)(sim_mat)
sim_mat_second_conv = Conv2D(filters=32*2, kernel_size=(3, 3),  activation='relu', padding='same',  strides=1)(sim_mat_first_conv)
sim_mat_third_conv = Conv2D(filters=32*3, kernel_size=(3, 3),  activation='relu', padding='same',  strides=1)(sim_mat_second_conv)
sim_mat_global_pooling = GlobalMaxPooling2D()(sim_mat_third_conv)


# network for drug similarities
drugs_sim_dense_1 = Conv1D(filters=32, kernel_size=3,  activation='relu', padding='same',  strides=1)
drugs_sim_do_1 = Dropout(0.1)
drugs_sim_dense_2 = Conv1D(filters=32*2, kernel_size=3,  activation='relu', padding='same',  strides=1)
drugs_sim_do_2 = Conv1D(filters=32*3, kernel_size=3,  activation='relu', padding='same',  strides=1)
drugs_sim_dense_3 = GlobalMaxPooling1D()


# Drug1
drug_sim_1 = drugs_sim_dense_1(Drug1_similarities)
drug_sim_1 = drugs_sim_do_1(drug_sim_1)
drug_sim_1 = drugs_sim_dense_2(drug_sim_1)
drug_sim_1 = drugs_sim_do_2(drug_sim_1)
drug_sim_1 = drugs_sim_dense_3(drug_sim_1)

# Drug2
drugs_sim_dense_1_ = Conv1D(filters=32, kernel_size=3,  activation='relu', padding='same',  strides=1)
drugs_sim_dense_2_ = Conv1D(filters=32*2, kernel_size=3,  activation='relu', padding='same',  strides=1)
drugs_sim_do_2_ = Conv1D(filters=32*3, kernel_size=3,  activation='relu', padding='same',  strides=1)
drugs_sim_dense_3_ = GlobalMaxPooling1D()
drugs_sim_do_1_ = Dropout(0.1)


drug_sim_2 = drugs_sim_dense_1_(Drug2_similarities)
drug_sim_2 = drugs_sim_do_1_(drug_sim_2)
drug_sim_2 = drugs_sim_dense_2_(drug_sim_2)
drug_sim_2 = drugs_sim_do_2_(drug_sim_2)
drug_sim_2 = drugs_sim_dense_3_(drug_sim_2)


# network for cell similarities
cell_sim_dense_1 = Dense(128)(cell_similarities)
cell_sim_do_1 = Dropout(0.2)(cell_sim_dense_1)
cell_sim_dense_2 = Dense(64, activation='relu')(cell_sim_do_1)
cell_sim_do_2 = Dropout(0.2)(cell_sim_dense_1)
cell_sim_dense_3 = Dense(32, activation='relu')(cell_sim_do_2)

encode_interaction = keras.layers.concatenate([encode_drug1, encode_drug2, cell_sim_dense_2 , sim_mat_global_pooling, drug_sim_1, drug_sim_2], axis=-1)#, cell_sim_dense_2, sim_mat_global_pooling, drug_sim_1, drug_sim_2
# Fully connected
FC1 = Dense(1024, kernel_regularizer=keras.regularizers.L1(1e-4))(encode_interaction)
FC2 = Dropout(0.1)(FC1)
"""FC2 = Dense(512, activation='relu')(FC2)
FC2 = Dropout(0.1)(FC2)
FC2 = Dense(256, activation='relu')(FC2)"""


# And add a logistic regression on top
predictions = Dense(1,  activation='sigmoid',  name='synergy')(FC2)
#Synergy = layers.Dense(1, name='synergy', activation='sigmoid')(features1)
# Create the Keras model.
model_synergy = keras.Model(inputs=[Drug1_input, Drug2_input,  cell_similarities, Drug1_similarities, Drug2_similarities],#, Drug1_similarities, Drug2_similarities],
                            outputs=[predictions])



METRICS = [
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR')  # precision-recall curve
]
adam = tf.optimizers.Adam(learning_rate=0.01)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=300,
    decay_rate=0.9)
adam = tf.optimizers.Adam(learning_rate=lr_schedule)

# adam=tf.optimizers.Adam(learning_rate=0.01)

model_synergy.compile(optimizer=adam, loss=['binary_crossentropy'], metrics={'synergy': [METRICS]})


indx_of_drugs1_combinations = np.array(indx_of_drugs1_combinations)
indx_of_drugs2_combinations = np.array(indx_of_drugs2_combinations)
drugs_combinations = np.array(drugs_combinations)

smiles_unique = np.array(smiles_unique)
indx_of_cells_combinations = np.array(indx_of_cells_combinations)

choice = np.random.choice(range(indx_of_drugs1_combinations.shape[0]),
                          size=(int(indx_of_drugs1_combinations.shape[0] * 0.8),), replace=False)

from sklearn.model_selection import KFold

kf = KFold(n_splits=10)

for i, (train_index, test_index) in enumerate(kf.split(indx_of_drugs1_combinations)):
    if i == 0:
      choice = np.array(list(train_index))

ind = np.zeros(indx_of_drugs1_combinations.shape[0], dtype=bool)
ind[choice] = True
rest = np.argwhere(~ind)

# print(choice.shape)
# print(rest.shape)

############################################
#  Splitting data into test and train      #
############################################

indx_of_drugs1_combinations_train = indx_of_drugs1_combinations[choice]
indx_of_drugs2_combinations_train = indx_of_drugs2_combinations[choice]
drugs_combinations_train = drugs_combinations[choice, :]
indx_of_cells_combinations_train = indx_of_cells_combinations[choice]

indx_of_drugs1_combinations_test = indx_of_drugs1_combinations[rest]
indx_of_drugs1_combinations_test = indx_of_drugs1_combinations_test.reshape(
    (indx_of_drugs1_combinations_test.shape[0],))
indx_of_drugs2_combinations_test = indx_of_drugs2_combinations[rest]
indx_of_drugs2_combinations_test = indx_of_drugs2_combinations_test.reshape(
    (indx_of_drugs2_combinations_test.shape[0],))
drugs_combinations_test = drugs_combinations[rest, :]
drugs_combinations_test = np.squeeze(drugs_combinations_test)
indx_of_cells_combinations_test = indx_of_cells_combinations[rest]
indx_of_cells_combinations_test = indx_of_cells_combinations_test.reshape((indx_of_cells_combinations_test.shape[0],))

# print(indx_of_cells_combinations_test.shape)
# print(drugs_combinations_train.shape)




def generate_data(batch_size, indx_of_drugs1_combinations, indx_of_drugs2_combinations,
                  indx_of_cells_combinations, drugs_combinations, num_training_samples, flag):
    i_c = 0
    drugs1 = []
    drugs2 = []
    combinations = []
    cells = []
    interactions1 = []
    interactions2 = []
    idx_perm = np.random.permutation(len(drugs_combinations))
    flag = 0
    while True:
        if i_c >= np.floor(num_training_samples / batch_size):
            i_c = 0
            flag = 0
            idx_perm = np.random.permutation(len(drugs_combinations))
        if flag == 0:
          drugs1 = smiles_unique[indx_of_drugs1_combinations[idx_perm[i_c * batch_size:(i_c + 1) * batch_size]]]
          drugs2 = smiles_unique[indx_of_drugs2_combinations[idx_perm[i_c * batch_size:(i_c + 1) * batch_size]]]
          drugs1_sim = tanimoto_sim_maccs[indx_of_drugs1_combinations[idx_perm[i_c * batch_size:(i_c + 1) * batch_size]]]
          drugs2_sim = tanimoto_sim_maccs[indx_of_drugs2_combinations[idx_perm[i_c * batch_size:(i_c + 1) * batch_size]]]

          combinations = drugs_combinations[idx_perm[i_c * batch_size:(i_c + 1) * batch_size]]
          idx = indx_of_cells_combinations[idx_perm[i_c * batch_size:(i_c + 1) * batch_size]]
          cell_sim = cell_similarities_[indx_of_cells_combinations[idx_perm[i_c * batch_size:(i_c + 1) * batch_size]]]
          

          i_c = i_c + 1
          flag = 1
          yield (drugs1, drugs2, cell_sim, drugs1_sim, drugs2_sim), combinations   #, drugs1_sim, drugs2_sim
        else:
          flag = 0
          yield (drugs2, drugs1, cell_sim, drugs2_sim, drugs1_sim), combinations


def generate_data_val(batch_size, indx_of_drugs1_combinations, indx_of_drugs2_combinations,
                      indx_of_cells_combinations, drugs_combinations, num_training_samples, flag):
    i_c = 0
    drugs1 = []
    drugs2 = []
    combinations = []
    cells = []
    interactions1 = []
    interactions2 = []
    while True:
        if i_c >= np.floor(num_training_samples / batch_size):
            i_c = 0
        drugs1 = smiles_unique[indx_of_drugs1_combinations[i_c * batch_size:(i_c + 1) * batch_size]]
        drugs2 = smiles_unique[indx_of_drugs2_combinations[i_c * batch_size:(i_c + 1) * batch_size]]
        drugs1_sim = np.reshape(tanimoto_sim_maccs[indx_of_drugs1_combinations[i_c * batch_size:(i_c + 1) * batch_size]],
                                    (batch_size, -1, 1))
        drugs2_sim = np.reshape(tanimoto_sim_maccs[indx_of_drugs2_combinations[i_c * batch_size:(i_c + 1) * batch_size]],
                                    (batch_size, -1, 1))
        combinations = drugs_combinations[i_c * batch_size:(i_c + 1) * batch_size]
        idx = indx_of_cells_combinations[i_c * batch_size:(i_c + 1) * batch_size]
        cell_sim = cell_similarities_[indx_of_cells_combinations[i_c * batch_size:(i_c + 1) * batch_size]]
        i_c = i_c + 1
        yield (np.expand_dims(drugs1, axis=-1), np.expand_dims(drugs2, axis=-1), cell_sim, drugs1_sim, drugs2_sim), combinations   #, drugs1_sim, drugs2_sim

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=300,
    decay_rate=0.9)
adam = tf.optimizers.Adam(learning_rate=lr_schedule)
es = EarlyStopping(monitor=adam, mode='min', verbose=1, patience=15)

model_checkpoint = ModelCheckpoint(
    filepath='best_model_without_sim_global.keras',
    monitor='val_auc',
    save_best_only=True,
    mode='max',
    save_weights_only=False
)

model_synergy.fit(
	generate_data(100, indx_of_drugs1_combinations_train, indx_of_drugs2_combinations_train,
				  indx_of_cells_combinations_train, drugs_combinations_train,
				  indx_of_drugs1_combinations_train.shape[0], 1),
	steps_per_epoch=int(np.floor(indx_of_drugs1_combinations_train.shape[0]/100))*2, epochs=100,
	validation_data=generate_data_val(10, indx_of_drugs1_combinations_test,
									  indx_of_drugs2_combinations_test, indx_of_cells_combinations_test,
									  drugs_combinations_test, indx_of_drugs1_combinations_test.shape[0], 1),
	validation_steps=int(np.floor(indx_of_drugs1_combinations_test.shape[0]/10)), callbacks=[model_checkpoint])


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

y_pred_prob = model_synergy.predict(generate_data_val(10, indx_of_drugs1_combinations_test,
									  indx_of_drugs2_combinations_test, indx_of_cells_combinations_test,
									  drugs_combinations_test, indx_of_drugs1_combinations_test.shape[0], 1), steps = int(np.floor(indx_of_drugs1_combinations_test.shape[0]/10)))

# Calculate ROC curve
y_test = drugs_combinations_test
fpr, tpr, _ = roc_curve(y_test[0:len(y_pred_prob)], y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()
